# Predicting Mortgage Rates from Government Data (FINAL)

### This code is developed as a part of the following Capstone competition for Microsoft MPP Data Scientist Certification:

https://www.datasciencecapstone.org/competitions/18/mortgage-rates-from-government-data/

### Training and test datasets are provided as a part of this repo.

### The purpose of the code is to calculate morgate rates given the government data. This is a regression problem.
### CatBoostRegressor machine learning model is used. Hyperparameter tuning using GridSearch with Cross Validation was also applied.

### Detailed report can be found in this [PDF document](REPORT_github_version.pdf).

#### Installing the required packages

In [ ]:
!pip install --upgrade pandas
!pip install --upgrade catboost
!pip install --upgrade matplotlib
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade seaborn


In [ ]:
 disable automatic scrolling of notebook output

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

#### Importing the required libraries. 

In [ ]:

import pandas as pd
import numpy as np
import matplotlib
import scipy
import seaborn as sns
import time,datetime

from scipy import stats

from matplotlib import pyplot
%matplotlib inline


## **Acquiring the data**

In [ ]:
# read in the training dataset
training_values = pd.read_csv('training_inputs.csv')
training_labels = pd.read_csv('training_labels.csv')
train = pd.merge(training_values,training_labels,on='row_id')

In [ ]:
# let us view it
train.head()

In [ ]:
# check how many rows and features (plus label) we have 
print(train.shape)

In [ ]:
# check for redundant rows
print(train.row_id.unique().shape)

#### FINDING: It seems that we have no redundant row_id entries in the training dataset.

In [ ]:
# check the data types for the training dataset
print(train.dtypes)

In [ ]:
# convert "co-applicant" from boolean to integer
train['co_applicant'] = train['co_applicant'].astype(int)

In [ ]:
# let us check it again
print(train.dtypes)

In [ ]:
# read in the test (validation) dataset
test = pd.read_csv('test_values.csv')

In [ ]:
# let us view the test (validation) dataset
test.head()

In [ ]:
# show how many rows and features we have - naturally, we do not have the label.
print(test.shape)

In [ ]:
# check for redundant rows
print(test.row_id.unique().shape)

#### *FINDING: It seems that we have no redundant row_id entries in the test dataset.*

In [ ]:
# check the data types for the test dataset
print(test.dtypes)

In [ ]:
# convert "co-applicant" to boolean integer
test['co_applicant'] = test['co_applicant'].astype(int)

In [ ]:
# check the data types for the training dataset
print(test.dtypes)

## **Capstone Challange 1 - Data Exploration**

### QUESTION : get basic stats for 'rate_spread'

In [ ]:
print('Rate Spread Min: %.1f' % (training_labels['rate_spread'].min()))
print('Rate Spread Max: %.1f' % (training_labels['rate_spread'].max()))
print('Rate Spread Mean: %.1f' % (training_labels['rate_spread'].mean()))
print('Rate Spread Median: %.1f' % (training_labels['rate_spread'].median()))
print('Rate Spread Std Dev: %.1f' % (training_labels['rate_spread'].std()))

### QUESTION: draw histogram for 'rate_spread'

In [ ]:

def plot_histogram(my_df, cols, bins = 200):
    for col in cols:
        fig = pyplot.figure(figsize=(6,6)) # define plot area
        ax = fig.gca() # define axis    
        my_df[col].plot.hist(ax = ax, bins = bins) # Use the plot.hist method on subset of the data frame
        ax.set_title('Histogram of ' + col) # Give the plot a main title
        ax.set_xlabel(col) # Set text for the x axis
        ax.set_ylabel('Frequency')# Set text for y axis
        pyplot.show()
        
num_cols = ['rate_spread']    
plot_histogram(train, num_cols)

### QUESTION : compare avg "rate_spread" between 'applicant_ethnicity = 1' vs 'applicant_ethnicity = 3'

In [ ]:
# let us use boxplot which gives us a lot of information including outliers
sns.boxplot(x='applicant_ethnicity',y='rate_spread',data=train)

In [ ]:
# printing the mean rate spread for "applicant_etnicity=1.0"
train[train.applicant_ethnicity == 1.0].rate_spread.mean()

In [ ]:
# printing the mean rate spread for "applicant_etnicity=3.0"
train[train.applicant_ethnicity == 3.0].rate_spread.mean()

#### FINDING: *Applicants where applicant_ethnicity=3 have a higher rate spread on average than where applicant_ethnicity=1.*

### QUESTION : compare avg "rate_spread" between 'sex = 1' vs 'sex = 2'

In [ ]:
# let us use boxplot which gives us a lot of information including outliers
sns.boxplot(x='applicant_sex',y='rate_spread',data=train)

In [ ]:
# printing the mean rate spread for "applicant_sex=1.0"
train[train.applicant_sex == 1.0].rate_spread.mean()

In [ ]:
# printing the mean rate spread for "applicant_sex=2.0"
train[train.applicant_sex == 2.0].rate_spread.mean()

#### FINDING: Applicants where applicant_sex=1 have a lower rate spread on average than where applicant_sex=2.*

### QUESTION: for applicants in state 43, get correlation between 'income' and 'loan amount'

In [ ]:
income = train[train.state_code == 43.0].applicant_income
loan = train[train.state_code == 43.0].loan_amount
corr_value = income.corr(loan)
print(corr_value)

In [ ]:
# now let us print the correlatin graph
pyplot.scatter(income,loan)
pyplot.xlabel('Applicant Income')
pyplot.ylabel('Loan Amount')
pyplot.show()

#### FINDING: *A higher applicant income is associated with a higher loan amount, on average.*

### QUESTION: limiting just to state 48 and ignoring where county is missing (missing value being -1) analyze 'rate_spread' for different counties

In [ ]:
# get the 'rate_spread' and 'county_code' for 'state_code == 48
temp_df = train[train.state_code == 48.0][['county_code','rate_spread']]

# drop the rows where 'county_code' = -1.0
temp_df.drop(temp_df[temp_df.county_code == -1.0].index, inplace=True)

# draw the boxplot
sns.boxplot(x='county_code',y='rate_spread',data=temp_df)

In state 48, the average rate spread across counties varies substantially, ranging from around 1% to around 7%.

### QUESTION: Looking just at states 2 and 3 and just loan types 1, 2, and 3, compare avg rate spread between states


In [ ]:
print(train.shape)
temp_df = train[((train.loan_type == 1.0) | (train.loan_type == 2.0) | (train.loan_type == 3.0))]
print(temp_df.shape)
temp_df2 = temp_df[(temp_df.state_code == 2.0) | (temp_df.state_code == 3.0)]
print(temp_df2.shape)

In [ ]:
# calculate the average rate spread in state 2 vs the overall rate spread among states 2 and 3.
m1 = temp_df2[temp_df2.state_code == 2.0].rate_spread.mean()
print('Avg rate spread for state 2: %.2f' % m1)

m2 = temp_df2.rate_spread.mean()
print('Avg rate spread for states 2 and 3: %.2f'  %m2)


In [ ]:
# calculate the average rate spread in state 3 vs the overall rate among states 2 and 3.
m1 = temp_df2[temp_df2.state_code == 3.0].rate_spread.mean()
print('Avg rate spread for state 3: %.2f' % m1)

m2 = temp_df2.rate_spread.mean()
print('Avg rate spread for states 2 and 3: %.2f'  %m2)

In [ ]:
# calculate the average rate spread in state 2 vs the overall rate in state 3

m1 = temp_df2[temp_df2.state_code == 2.0].rate_spread.mean()
print('Avg rate spread for state 2: %.2f' % m1)

m2 = temp_df2[temp_df2.state_code == 3.0].rate_spread.mean()
print('Avg rate spread for state 3: %.2f'  %m2)

For loan types 1, 2, and 3, the average rate spread in state 2 is higher than for the corresponding loan type in state 3. 

## **Feature Engineering**

### Remove outlier for dependent value

In [ ]:
# let us see the outliers in a boxplot
sns.boxplot(y='rate_spread',data=train)

In [ ]:
# CUT OFF VALUE FOR TRAINING DATASET
cut_off = 50.0

In [ ]:
# let us check the number of outlier values
train[(train['rate_spread']>cut_off)].count()

In [ ]:
# applying the cutoff
train = train[train.rate_spread < cut_off]

In [ ]:
train.shape

### Impute missing values for training dataset

In [ ]:
# we are given the information that a value of '-1' indicates missing value for the following columns:
# msa_md, state_code, county_code
# if such a value exists, we will replace it with NaN
train['msa_md'].replace(-1.0, np.NaN,inplace=True)
train['state_code'].replace(-1.0, np.NaN,inplace=True)
train['county_code'].replace(-1.0, np.NaN,inplace=True)

In [ ]:
train.isna().sum()

In [ ]:
# let us see what percentage of the total data is missing
def missing_values_table(df):
        mis_val = df.isna().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(train)

In [ ]:
# now it is time to impute missing values
# if a feature is numeric, then use the mean()
# for categorical features, use mode()
train['state_code'].fillna(train['state_code'].mode()[0], inplace=True)
train['applicant_income'].fillna(train['applicant_income'].mean(), inplace=True)
train['population'].fillna(train['population'].mean(), inplace=True)
train['minority_population_pct'].fillna(train['minority_population_pct'].mean(), inplace=True)
train['ffiecmedian_family_income'].fillna(train['ffiecmedian_family_income'].mean(), inplace=True)
train['tract_to_msa_md_income_pct'].fillna(train['tract_to_msa_md_income_pct'].mean(), inplace=True)
train['number_of_owner-occupied_units'].fillna(train['number_of_owner-occupied_units'].mean(), inplace=True)
train['number_of_1_to_4_family_units'].fillna(train['number_of_1_to_4_family_units'].mean(), inplace=True)

In [ ]:
# check for null values in the training dataset after imputing
train.isna().sum()

### Impute missing values for testing dataset

In [ ]:
# we are given that a value of '-1' indicates missing value for the following columns:
# msa_md, state_code, county_code
# if such a value exists, we will replace it with NaN
test['msa_md'].replace(-1.0, np.NaN,inplace=True)
test['state_code'].replace(-1.0, np.NaN,inplace=True)
test['county_code'].replace(-1.0, np.NaN,inplace=True)

In [ ]:
# check for null values in the test dataset
test.isna().sum()

In [ ]:
missing_values_table(test)

In [ ]:
# we are given that a value of '-1' indicates missing value for the following columns:
# msa_md, state_code, county_code
# if such a value exists, we will replace it with NaN
test['state_code'].fillna(test['state_code'].mode()[0], inplace=True)
test['applicant_income'].fillna(test['applicant_income'].mean(), inplace=True)
test['population'].fillna(test['population'].mean(), inplace=True)
test['minority_population_pct'].fillna(test['minority_population_pct'].mean(), inplace=True)
test['ffiecmedian_family_income'].fillna(test['ffiecmedian_family_income'].mean(), inplace=True)
test['tract_to_msa_md_income_pct'].fillna(test['tract_to_msa_md_income_pct'].mean(), inplace=True)
test['number_of_owner-occupied_units'].fillna(test['number_of_owner-occupied_units'].mean(), inplace=True)
test['number_of_1_to_4_family_units'].fillna(test['number_of_1_to_4_family_units'].mean(), inplace=True)

In [ ]:
# check for null values in the test dataset after imputing
test.isna().sum()

## **Exploratory Data Analysis (EDA)**

#### Displaying the statistics of the training dataset

In [ ]:
# display statistics
train.describe()

#### *FINDING: The mean() values and minx/max ranges differ between features. We need to normalize these features.*

In [ ]:
# lets recall the data types
print(train.dtypes)

### Creating histograms of numeric features

In [ ]:

def plot_histogram_with_overlay(the_array, x_title, the_flag):
    
    pyplot.hist(the_array, alpha=0.5, bins=50, density=True)

    if(the_flag==True):
        # find minimum and maximum of xticks, so we know
        # where we should compute theoretical distribution
        xt = pyplot.xticks()[0]  
        xmin, xmax = min(xt), max(xt)  
        lnspc = np.linspace(xmin, xmax, len(the_array))

        m, s = stats.norm.fit(the_array) # get mean and standard deviation  
        pdf_g = stats.norm.pdf(lnspc, m, s) # now get theoretical values in our interval  
        # Plot some fancy text to show us what the parameters of the distribution are (mean and standard deviation)
        pyplot.text(x=np.min(the_array), y=0.1, s=r"$\mu=%0.1f$" % m + "\n" + r"$\sigma=%0.1f$" % s, color='r')
        pyplot.plot(lnspc, pdf_g, label="Normal") # plot it

    # Standard plot stuff
    pyplot.xlabel(x_title)
    pyplot.title('Histogram of ' + x_title )
    pyplot.show()
        

In [ ]:
pyplot.rcParams["figure.figsize"]=5,5

In [ ]:
plot_histogram_with_overlay(train['loan_amount'], "Loan Amount",False)

In [ ]:
# print skewness value
print(train['loan_amount'].skew())

#### *FINDING: Significant right skew is observed for the loan_amount.*

In [ ]:
plot_histogram_with_overlay(train['applicant_income'], "Applicant_income", False)

In [ ]:
# print skewness value
print(train['applicant_income'].skew())

#### *FINDING: Significant right skew is observed for the applicant_income.*

In [ ]:
plot_histogram_with_overlay(train['population'], "Population",False)

In [ ]:
# print skewness value
print(train['population'].skew())

#### *FINDING: Significant right skew is observed for the population.*

In [ ]:
plot_histogram_with_overlay(train['minority_population_pct'], "minority_population_pct",False)

#### *FINDING: The histogram for minority_population_pct seems to be exponential.*

In [ ]:
plot_histogram_with_overlay(train['ffiecmedian_family_income'], "ffiecmedian_family_income",False)

In [ ]:
# print skewness value
print(train['ffiecmedian_family_income'].skew())

#### *FINDING: Some small right skew is observed for ffiecmedian_family_income.*

In [ ]:
plot_histogram_with_overlay(train['tract_to_msa_md_income_pct'], "tract_to_msa_md_income_pct",False)

In [ ]:
# print skewness value
print(train['tract_to_msa_md_income_pct'].skew())

#### *FINDING: Significant left skew is observed for tract_to_msa_md_income_pct.*

In [ ]:
plot_histogram_with_overlay(train['number_of_owner-occupied_units'], "number_of_owner-occupied_units",False)

In [ ]:
# print skewness value
print(train['number_of_owner-occupied_units'].skew())

#### *FINDING: Some right skew is observed for number_of_owner-occupied_units.*

### Analyzing Categorical features

In [ ]:
def frequency_table(col_name):
    # Get the value counts and percentages for the specific column
    value_counts = train[col_name].value_counts().sort_index()
    percentages = train[col_name].value_counts(normalize=True).sort_index() * 100
    percentages = percentages.round(2) # Limit to 2 digits of accuracy


    # Concatenate the value counts and percentages into a DataFrame
    table = pd.concat([value_counts, percentages], axis=1)
    table.columns = ['count', 'percentage']

    # Print the table
    print(table)

In [ ]:
# Checking the frequency distribution of 'loan type' 
# 1 -- Conventional (any loan other than FHA, VA, FSA, or RHS loans)
# 2 -- FHA-insured (Federal Housing Administration)
# 3 -- VA-guaranteed (Veterans Administration)
# 4 -- FSA/RHS (Farm Service Agency or Rural Housing Service)

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="loan_type", data=train);

In [ ]:
frequency_table('loan_type')

#### *FINDING: It seems that most of the loans are "FHA-insured".*

In [ ]:
# Checking the frequency distribution of 'property type' 
# 1 -- One to four-family (other than manufactured housing)
# 2 -- Manufactured housing
# 3 -- Multifamily

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="property_type", data=train);

In [ ]:
frequency_table('property_type')

#### *FINDING: It seems that '1-4 family' homes are most financed. Also there are no multifamily entries in the training dataset.*

In [ ]:
# Checking the frequency distribution of 'loan purpose' 
# 1 -- Home purchase
# 2 -- Home improvement
# 3 -- Refinancing

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="loan_purpose", data=train);

In [ ]:
frequency_table('loan_purpose')

#### *FINDING: Most loans are for home purchase.*

In [ ]:
# Checking the frequency distribution of 'occupancy' 
# 1 -- Owner-occupied as a principal dwelling
# 2 -- Not owner-occupied
# 3 -- Not applicable

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="occupancy", data=train);

In [ ]:
frequency_table('occupancy')

#### *FINDING: Most loans are for principal dwelling.*

In [ ]:
# Checking the frequency distribution of 'preapproval'  requirement
# 1 -- Preapproval was requested
# 2 -- Preapproval was not requested
# 3 -- Not applicable

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="preapproval", data=train);

In [ ]:
frequency_table('preapproval')

#### *FINDING: Preapproval was not required for most loans.*

In [ ]:
# Checking the frequency distribution of 'applicant ethnicity'
# 1 -- Hispanic or Latino
# 2 -- Not Hispanic or Latino
# 3 -- Information not provided by applicant in mail, Internet, or telephone application
# 4 -- Not applicable
# 5 -- No co-applicant

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="applicant_ethnicity", data=train);

In [ ]:
frequency_table('applicant_ethnicity')

#### *FINDING: Mortgages were rewarded mostly to "not hispanic or latino"*

In [ ]:
# Checking the frequency distribution of 'applicant race'
# 1 -- American Indian or Alaska Native
# 2 -- Asian
# 3 -- Black or African American
# 4 -- Native Hawaiian or Other Pacific Islander
# 5 -- White
# 6 -- Information not provided by applicant in mail, Internet, or telephone application
# 7 -- Not applicable
# 8 -- No co-applicant

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="applicant_race", data=train);

In [ ]:
frequency_table('applicant_race')

#### *FINDING: Loan approvals were overwhelmingly granted to whites.*
#### *FINDING: There are no rows where applicant_race = 8.0*

In [ ]:
# Checking the frequency distribution of 'applicant sex'
# 1 -- Male
# 2 -- Female
# 3 -- Information not provided by applicant in mail, Internet, or telephone application
# 4 or 5 -- Not applicable

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="applicant_sex", data=train);

In [ ]:
frequency_table('applicant_sex')

#### *FINDING: Most loans were granted to males.* 


#### *FINDING: There are no rows where applicant_sex = 5.0.*

In [ ]:
# Checking the frequency distribution of 'co applicant'
# 0 -- no spouse
# 1 -- spouse

# Let's view the distribution
pyplot.figure(figsize=(20, 5)) 
sns.countplot(y="co_applicant", data=train);

In [ ]:
frequency_table('co_applicant')

#### *FINDING: Almost twice as many mortgages are granted to no-spouse applicants (single applicants).*

#### **Relationships between the categorical features and the dependent variable**

In [ ]:
# Checking the frequency distribution of 'loan type' 
# 1 -- Conventional (any loan other than FHA, VA, FSA, or RHS loans)
# 2 -- FHA-insured (Federal Housing Administration)
# 3 -- VA-guaranteed (Veterans Administration)
# 4 -- FSA/RHS (Farm Service Agency or Rural Housing Service)

pyplot.figure(figsize=(10, 10)) 
sns.boxplot(x='loan_type',y='rate_spread',data=train)

#### *FINDING: The average rate spread is higher for conventional loans.*

In [ ]:
# Checking the frequency distribution of 'property type' 
# 1 -- One to four-family (other than manufactured housing)
# 2 -- Manufactured housing
# 3 -- Multifamily

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='property_type',y='rate_spread',data=train)

#### *FINDING: One-to-four family loans have the lowest average rate_spread*

In [ ]:
# Checking the frequency distribution of 'loan purpose' 
# 1 -- Home purchase
# 2 -- Home improvement
# 3 -- Refinancing

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='loan_purpose',y='rate_spread',data=train)

#### *FINDING: Home reimprovement loans have a higher average rate spread.*

In [ ]:
# Checking the frequency distribution of 'occupancy' 
# 1 -- Owner-occupied as a principal dwelling
# 2 -- Not owner-occupied
# 3 -- Not applicable

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='occupancy',y='rate_spread',data=train)

#### *FINDING: Unknown or "not applicable" occupancy has the highest average loan spread.*

In [ ]:
# Checking the frequency distribution of 'preapproval'  requirement
# 1 -- Preapproval was requested
# 2 -- Preapproval was not requested
# 3 -- Not applicable

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='preapproval',y='rate_spread',data=train)

#### *FINDING: Preapproval was not a significant requirement.*

In [ ]:
# Checking the frequency distribution of 'applicant ethnicity'
# 1 -- Hispanic or Latino
# 2 -- Not Hispanic or Latino
# 3 -- Information not provided by applicant in mail, Internet, or telephone pplication
# 4 -- Not applicable
# 5 -- No co-applicant

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='applicant_ethnicity',y='rate_spread',data=train)

#### *FINDING: If ethnicity information is not provided by the applicant, the average rate spread is much higher. THIS IS VERY ALARMING.*

In [ ]:
# Checking the frequency distribution of 'applicant race'
# 1 -- American Indian or Alaska Native
# 2 -- Asian
# 3 -- Black or African American
# 4 -- Native Hawaiian or Other Pacific Islander
# 5 -- White
# 6 -- Information not provided by applicant in mail, Internet, or telephone application
# 7 -- Not applicable
# 8 -- No co-applicant

pyplot.figure(figsize=(10,15)) 
sns.boxplot(x='applicant_race',y='rate_spread',data=train)

#### *FINDING: Average rate spread is much higher for "American Indian or Alaska Natives".*
#### *FINDING: Similar to ethnicity, if race information is not provided by the applicant, the standard deviation for rate spread is much greater than others.*

In [ ]:
# Checking the frequency distribution of 'applicant sex'
# 1 -- Male
# 2 -- Female
# 3 -- Information not provided by applicant in mail, Internet, or telephone application
# 4 or 5 -- Not applicable

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='applicant_sex',y='rate_spread',data=train)

#### *FINDING: While the average rate spread is pretty much equal, if the applicant does not specify gender, then the standard deviation of rate spread is higher.* 

In [ ]:
# Checking the frequency distribution of 'co applicant'
# 0 -- no spouse
# 1 -- spouse

pyplot.figure(figsize=(10,10)) 
sns.boxplot(x='co_applicant',y='rate_spread',data=train)

#### *FINDING: There is not much difference in resulting rate spread for different co applicant types.*

## Feature Engineering

### Transforming  the training dataset

In [ ]:
print(train.dtypes)

#### Applying Log transform

In [ ]:
# we will apply log transformation to 'loan_amount' since its histogram had skew
# we are adding a new feature
train['log_of_loan_amount'] = np.log(train['loan_amount'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_loan_amount'], "Log of loan_amount", True)

In [ ]:
# we will apply log transformation to 'applicant_income' since its histogram had skew
# we are adding a new feature
train['log_of_applicant_income'] = np.log(train['applicant_income'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_applicant_income'], "Log of applicant_income",True)

In [ ]:
# we will apply log transformation to 'population' since its histogram had skew
# we are adding a new feature
train['log_of_population'] = np.log(train['population'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_population'], "Log of Population",True)

#### Applying square root transformation (Boxcox)

In [ ]:
from scipy.stats import boxcox
# we will apply square root transformation to 'minority_population_pct' 
# since its histogram had a decaying exponential skew
# we are adding a new feature
train['sqrt_of_minority_population_pct'] = boxcox(train['minority_population_pct'],0.5)
# check the histogram after log transformation
plot_histogram_with_overlay(train['sqrt_of_minority_population_pct'], "Sqrt of minority_population_pct",True)

In [ ]:
# we will apply log transformation to 'ffiecmedian_family_income' since its histogram had skew
# we are adding a new feature
train['log_of_ffiecmedian_family_income'] = np.log(train['ffiecmedian_family_income'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_ffiecmedian_family_income'], "log_of_ffiecmedian_family_income",True)

In [ ]:
# we will apply log transformation to 'tract_to_msa_md_income_pct' since its histogram had skew
# we are adding a new feature
train['log_of_tract_to_msa_md_income_pct'] = np.log(train['tract_to_msa_md_income_pct'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_tract_to_msa_md_income_pct'], "log_of_tract_to_msa_md_income_pct",True)

In [ ]:
# we will apply log transformation to 'number_of_owner-occupied_units' since its histogram had skew
# we are adding a new feature
train['log_of_number_of_owner-occupied_units'] = np.log(train['number_of_owner-occupied_units'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_number_of_owner-occupied_units'], "log_of_number_of_owner-occupied_units",True)

In [ ]:
# we will apply log transformation to 'number_of_1_to_4_family_units' since its histogram had skew
# we are adding a new feature
train['log_of_number_of_1_to_4_family_units'] = np.log(train['number_of_1_to_4_family_units'])
# check the histogram after log transformation
plot_histogram_with_overlay(train['log_of_number_of_1_to_4_family_units'], "log_of_number_of_1_to_4_family_units",True)

#### Binning the training dataset

In [ ]:
# binning msa_md
def bin_msa_md(x):
    if 49 >= x >= 0: return 0
    elif 99 >= x >= 50: return 1
    elif 149 >= x >= 100: return 2
    elif 199 >= x >= 150: return 3
    elif 249 >= x >= 200: return 4
    elif 299 >= x >= 250: return 5
    elif 349 >= x >= 300: return 6
    elif 399 >= x >= 350: return 7
    elif 449 >= x >= 400: return 8
    else : return 9

In [ ]:
train['msa_md_group'] = train['msa_md'].map(bin_msa_md)

In [ ]:
plot_histogram_with_overlay(train['msa_md_group'], "msa_md_group",False)

In [ ]:
plot_histogram_with_overlay(train['county_code'], "county_code",False)

In [ ]:
# binning county_code

def bin_county_code(x):
    if 49 >= x >= 0: return 0
    elif 99 >= x >= 50: return 1
    elif 149 >= x >= 100: return 2
    elif 199 >= x >= 150: return 3
    elif 249 >= x >= 200: return 4
    elif 299 >= x >= 250: return 5
    elif 349 >= x >= 300: return 6
    elif 399 >= x >= 350: return 7
    elif 449 >= x >= 400: return 8
    else : return 9

In [ ]:
train['county_group'] = train['county_code'].map(bin_county_code)

In [ ]:
plot_histogram_with_overlay(train['county_group'], "county_group",False)

In [ ]:
plot_histogram_with_overlay(train['lender'], "lender",False)

In [ ]:
# binning lender

def bin_lender(x):
    if 499 >= x >= 0: return 0
    elif 999 >= x >= 500: return 1
    elif 1499 >= x >= 1000: return 2
    elif 1999 >= x >= 1500: return 3
    elif 2499 >= x >= 2000: return 4
    elif 2999 >= x >= 2500: return 5
    elif 3499 >= x >= 3000: return 6
    elif 3999 >= x >= 3500: return 7
    elif 4499 >= x >= 4000: return 8
    else : return 9

In [ ]:
train['lender_group'] = train['lender'].map(bin_lender)

In [ ]:
plot_histogram_with_overlay(train['lender_group'], "lender_group",False)

### Setting the data types for categorical features of the training dataset

In [ ]:
# check the resulting data types for training dataset
print(train.dtypes)

### Transforming the test dataset

In [ ]:
plot_histogram_with_overlay(test['loan_amount'], "loan_amount",False)

In [ ]:
test['log_of_loan_amount'] = np.log(test['loan_amount'])
plot_histogram_with_overlay(test['log_of_loan_amount'], "Log of loan_amount",True)

In [ ]:
plot_histogram_with_overlay(test['applicant_income'], "applicant_income",False)

In [ ]:
test['log_of_applicant_income'] = np.log(test['applicant_income'])
plot_histogram_with_overlay(test['log_of_applicant_income'], "Log of applicant_income",True)

In [ ]:
plot_histogram_with_overlay(test['population'], "population",False)

In [ ]:
test['log_of_population'] = np.log(test['population'])
plot_histogram_with_overlay(test['log_of_population'], "Log of Population",True)

In [ ]:
plot_histogram_with_overlay(test['minority_population_pct'], "minority_population_pct",False)

In [ ]:
from scipy.stats import boxcox
test['sqrt_of_minority_population_pct'] = boxcox(test['minority_population_pct'],0.5)
plot_histogram_with_overlay(test['sqrt_of_minority_population_pct'], "Sqrt of minority_population_pct",True)

In [ ]:
plot_histogram_with_overlay(test['ffiecmedian_family_income'], "ffiecmedian_family_income",False)

In [ ]:
test['log_of_ffiecmedian_family_income'] = np.log(test['ffiecmedian_family_income'])
plot_histogram_with_overlay(test['log_of_ffiecmedian_family_income'], "log_of_ffiecmedian_family_income",True)

In [ ]:
plot_histogram_with_overlay(test['tract_to_msa_md_income_pct'], "tract_to_msa_md_income_pct",False)

In [ ]:
test['log_of_tract_to_msa_md_income_pct'] = np.log(test['tract_to_msa_md_income_pct'])
plot_histogram_with_overlay(test['log_of_tract_to_msa_md_income_pct'], "log_of_tract_to_msa_md_income_pct",True)

In [ ]:
plot_histogram_with_overlay(test['number_of_owner-occupied_units'], "number_of_owner-occupied_units",False)

In [ ]:
test['log_of_number_of_owner-occupied_units'] = np.log(test['number_of_owner-occupied_units'])
plot_histogram_with_overlay(test['log_of_number_of_owner-occupied_units'], "log_of_number_of_owner-occupied_units",True)

In [ ]:
plot_histogram_with_overlay(train['number_of_1_to_4_family_units'], "number_of_1_to_4_family_units",False)

In [ ]:
test['log_of_number_of_1_to_4_family_units'] = np.log(test['number_of_1_to_4_family_units'])
plot_histogram_with_overlay(train['log_of_number_of_1_to_4_family_units'], "log_of_number_of_1_to_4_family_units",True)

### Binning the test dataset

In [ ]:
test['msa_md_group'] = test['msa_md'].map(bin_msa_md)
test['county_group'] = test['county_code'].map(bin_county_code)
test['lender_group'] = test['lender'].map(bin_lender)

### Display correlation matrix

In [ ]:
pyplot.rcParams["figure.figsize"]=30,30

corr = train.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 1.0})
# pyplot.figure(figsize=(15,8))
a = sns.heatmap(corr, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)
pyplot.savefig('corr.png')

#### The higher correlations are shown as white.

### Setting the data types for categorical features of the training and test datasets for the Cat Boost Algorithm

In [ ]:
# check the urrent data types for training dataset
print(train.dtypes)

In [ ]:
train = train.astype({"loan_type": str, 
                    "property_type": str,
                    "loan_purpose": str,
                    "occupancy": str,
                    "preapproval": str,
                    "msa_md": str,
                    "state_code": str,
                    "county_code": str,
                    "applicant_ethnicity": str,
                    "applicant_race": str,
                    "applicant_sex": str,
                    "lender": str,
                    "co_applicant": str,
                    "msa_md_group" : str,
                    "county_group" : str,
                    "lender_group" : str
                   })

In [ ]:
# check the resulting data types for training dataset
print(train.dtypes)

In [ ]:
# check the current data types for test dataset
print(test.dtypes)

In [ ]:
test = test.astype({"loan_type": str, 
                    "property_type": str,
                    "loan_purpose": str,
                    "occupancy": str,
                    "preapproval": str,
                    "msa_md": str,
                    "state_code": str,
                    "county_code": str,
                    "applicant_ethnicity": str,
                    "applicant_race": str,
                    "applicant_sex": str,
                    "lender": str,
                    "co_applicant": str,
                    "msa_md_group" : str,
                    "county_group" : str,
                    "lender_group" : str
                   })

In [ ]:
# check the resulting data types for test dataset
print(test.dtypes)

### Create the features matrix and target array

In [ ]:
#Creating a training set for modeling and validation set to check model performance
X_train = train.drop(['row_id','loan_amount','applicant_income','population','minority_population_pct',
                      'ffiecmedian_family_income','tract_to_msa_md_income_pct',
                      'number_of_owner-occupied_units','number_of_1_to_4_family_units',
#                     'msa_md',
#                      'msa_md_group'
#                    'county_code',
#                      'country_group'
#                      'lender',
#                      'lender_group'
                      'rate_spread'], axis=1)
y_train = train.rate_spread

X_test = test.drop(['row_id','loan_amount','applicant_income','population','minority_population_pct',
                      'ffiecmedian_family_income','tract_to_msa_md_income_pct',
                      'number_of_owner-occupied_units','number_of_1_to_4_family_units',
#                     'msa_md',
#                      'msa_md_group'
#                    'county_code',
#                      'country_group'
#                      'lender',
#                      'lender_group'
#                     'lender'
                   ], axis=1)

In [ ]:
X_train.dtypes

In [ ]:
X_test.dtypes

### Model scoring using Cross Validation

In [ ]:
# this is needed for the Cat Boost model
categorical_features_indices = np.where(X_train.dtypes == object)[0]
print(categorical_features_indices)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV


# define the model and set the initial parameters for CatBoostRegressor model
model=CatBoostRegressor(iterations=200, depth=6, learning_rate=0.1, loss_function='RMSE')


# set the parameters for KFold Cross Validation
num_folds = 3
random_seed = 1234
scoring_metric = 'r2'

kfold = KFold(n_splits=num_folds, shuffle=True,random_state=random_seed)


start_time = time.time()

cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring_metric, 
                             fit_params={'cat_features':categorical_features_indices,'logging_level': 'Silent'})

task_duration = (time.time() - start_time)


msg = "Mean: %f \t StdDev: (%f)" % (cv_results.mean(), cv_results.std())
print(msg)

print("CV Scoring - Running Time: {}".format(datetime.timedelta(seconds=task_duration)))

### Hyperparameter tuning using GridSearch with Cross Validation

In [ ]:

# set the parameter search grid
parameters = {'depth'         : [6,10,12],
              'learning_rate' : [0.1,0.2],
              'iterations'    : [200, 400,800]
             }


kfold = KFold(n_splits=num_folds, shuffle=True,random_state=random_seed)

start_time = time.time()

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = kfold, n_jobs=-1)
grid.fit(X_train, y_train,cat_features=categorical_features_indices,verbose=False)

task_duration = (time.time() - start_time)

print("GridSearchCV Parameter Tuning - Running Time: {}".format(datetime.timedelta(seconds=task_duration)))

In [ ]:
# Display the results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",grid.best_score_)

print("\n The best parameters across ALL searched params:\n", grid.best_params_)
    
print("\n ========================================================")

In [ ]:
# store the best parameter to final training of the model
optimal_parameters = grid.best_params_  

In [ ]:
# train the model with optimal parameters

best_model = CatBoostRegressor(iterations=optimal_parameters["iterations"],depth=optimal_parameters["depth"],
                          learning_rate=optimal_parameters["learning_rate"], loss_function='RMSE')


best_model.fit(X_train,y_train,cat_features=categorical_features_indices,verbose=False)

In [ ]:
# Feature Importance
def feature_importance(model, data):
    """
    Function to show which features are most important in the model.
    ::param_model:: Which model to use?
    ::param_data:: What data to use?
    """
    fea_imp = pd.DataFrame({'imp': model.feature_importances_, 'col': data.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
    _ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
    return fea_imp

In [ ]:
# displlay information on feature importance
feature_importance(best_model, X_train)

### Prediction and submission of results

In [ ]:
# calculate the model predictions using the model with optimally tuned parameters
predictions = best_model.predict(X_test)

In [ ]:
# create a dataframe for submission
submission = pd.DataFrame(data=predictions, columns=['rate_spread'])
submission['row_id'] = test['row_id']

In [ ]:
# save the results to CSV file
header = ['row_id','rate_spread']
submission.to_csv('predicting_mortgage_rates_FINAL.csv', columns=header, index=False)

##### The resulting CSV file was uploaded to public site of the competition. We get a score of 0.76 which No:12 in the public scoreboard.